# Exemple consumer Kafka avec PySpark

 ## Consumer avec Spark

Apache Spark est un moteur de traitement distribué écrit en Scala, langage s’exécutant sur la JVM (Java Virtual Machine).
Il permet de manipuler de grands volumes de données en mémoire, en mode batch ou streaming.

PySpark est l’interface Python officielle de Spark.
Elle agit comme un pont : les instructions Python sont traduites en requêtes Spark exécutées sur la JVM.
Ainsi, le développeur Python bénéficie de la puissance de Spark sans écrire de code Scala ou Java.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import time

# ... (Constantes et dépendances) ...
SPARK_VERSION = "4.0.1" 
BROKER_ADDRESS = "broker:29092"
TOPIC = "test" 
STARTING_OFFSET_MODE = "earliest" 
TABLE_NAME = "kafka_data_stream" 
KAFKA_CONNECTOR_VERSION = "4.0.0" 
KAFKA_CONNECTOR_PACKAGE = f"org.apache.spark:spark-sql-kafka-0-10_2.13:{KAFKA_CONNECTOR_VERSION}"
COMMON_KAFKA_PACKAGE = "org.apache.kafka:kafka-clients:3.7.0" 

# -------- Dépendances --------------------
spark = SparkSession.builder \
    .appName("KafkaConsumerStream") \
    .config("spark.jars.packages", f"{KAFKA_CONNECTOR_PACKAGE},{COMMON_KAFKA_PACKAGE}") \
    .getOrCreate()

# *** AJOUT DE LA LOGIQUE D'ARRÊT ***
for s in spark.streams.active:
    if s.name == TABLE_NAME and s.isActive:
        print(f"Arrêt du stream existant: {TABLE_NAME}")
        s.stop()
        # Optionnel: attendre un court instant pour la propreté
        time.sleep(1) 
# *** FIN DE LA LOGIQUE D'ARRÊT ***

spark.sparkContext.setLogLevel("WARN")

# -------- Connexion à Kafka ----------------
print(f"Démarrage de la lecture depuis le topic '{TOPIC}' avec l'offset: {STARTING_OFFSET_MODE}")

# ... (Définition du DataFrame df) ...
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", BROKER_ADDRESS) \
    .option("startingOffsets", STARTING_OFFSET_MODE) \
    .option("subscribe", TOPIC) \
    .load() \
    .selectExpr(
        "CAST(key AS STRING) AS key", 
        "CAST(value AS STRING) AS value",
        "topic",
        "partition",
        "offset",
        "timestamp"
    )

# Affichage des résultats dans une TABLE EN MÉMOIRE
query = df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName(TABLE_NAME) \
    .start()

print(f"✅ Le stream est démarré et actif. Nom: {query.name}")

## Boucle pour affichage

Affichage du dataframe précédent.

5 boucles de 3 secondes 

In [ ]:
# Cellule pour afficher les résultats
# La commande 'display' est souvent disponible dans les environnements Jupyter/Databricks
from IPython.display import display, clear_output
import time

# Boucle pour rafraîchir l'affichage
for x in range(5): # Affichera pendant 30 cycles (ex: 90 secondes)
    clear_output(wait=True)
    
    # Requête pour lire la table mise à jour par le stream
    results_df = spark.sql(f"SELECT * FROM {TABLE_NAME}")
    
    # Affichage des résultats (utilise show() pour une sortie texte simple)
    results_df.show(truncate=False) 
    
    # Si vous voulez afficher sous forme de tableau interactif (selon votre environnement)
    # display(results_df) 
    
    time.sleep(3) # Attendre 3 secondes

# Affichage de fin
print("Affichage du flux terminé ou arrêté.")
query.stop() # Arrête le stream proprement